In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from sklearn.preprocessing import LabelEncoder
import mne

In [8]:
mne_data = mne.read_epochs('beta.fif')
mne_data

Reading /home/jesse/UTFPR/RECONHECIMENTO_DE_PADROES/tutoriais/SSVEP/src/beta/beta.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1462.89 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 1.463 s
Baseline,off


In [5]:
y = np.load("../../datasets/beta/labels.npy")

print("y", y)
y_set = sorted(set(y))
y_set

y [ 8.6  8.8  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2
 11.4 11.6 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.
 14.2 14.4 14.6 14.8 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8
  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6
 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4
 14.6 14.8 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8  9.   9.2
  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.
 12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8
 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8  9.   9.2  9.4  9.6
  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.  12.2 12.4
 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8 15.  15.2
 15.4 15.6 15.8  8.   8.2  8.4]


[8.0,
 8.2,
 8.4,
 8.6,
 8.799999999999999,
 9.0,
 9.2,
 9.4,
 9.6,
 9.8,
 10.0,
 10.2,
 10.4,
 10.6,
 10.8,
 11.0,
 11.2,
 11.4,
 11.6,
 11.8,
 12.0,
 12.2,
 12.4,
 12.600000000000001,
 12.8,
 13.0,
 13.200000000000001,
 13.4,
 13.600000000000001,
 13.8,
 14.0,
 14.200000000000001,
 14.4,
 14.600000000000001,
 14.8,
 15.0,
 15.200000000000001,
 15.4,
 15.600000000000001,
 15.8]

In [11]:

import warnings
warnings.filterwarnings('ignore')

limiar = 0.2

hits = 0


for i in range(len(mne_data)):
    list_frequency = np.array([])
    dict_y = {str(value): index  for index, value in enumerate(y_set)}

    for frequency in y_set:
        high_frequency = frequency + limiar
        low_frequency = frequency - limiar
        print(f"Frequency: {frequency} -> Lower: {low_frequency} Max: {high_frequency}")

        view = mne_data[i].compute_psd(method='multitaper', fmin=low_frequency, fmax=high_frequency, verbose=False)
        array_values = np.array((view)[0][0])

        # value_view = np.max(array_values)
        value_view = np.average(array_values)
        # value_view = np.median(array_values)

        dict_y[str(frequency)] = value_view

    target_frequency = max(dict_y, key=dict_y.get)
    if float(target_frequency) == float(list(mne_data[i].event_id)[0]):
        hits += 1


<EpochsFIF |  1 events (all good), 0 – 1.46289 s, baseline off, ~444 kB, data loaded,
 '8.6': 1>
Frequency: 8.0 -> Lower: 7.8 Max: 8.2
Frequency: 8.2 -> Lower: 7.999999999999999 Max: 8.399999999999999
Frequency: 8.4 -> Lower: 8.200000000000001 Max: 8.6


ValueError: zero-size array to reduction operation minimum which has no identity

In [9]:
print(f"Percentage: {hits*100/len(y)}")


Percentage: 80.0
